In [19]:
def ModExp(a,x,p):
    if type(a)!=int or type(x)!=int or type(p)!=int:
        return TypeError('Inputs must be integers.')
    if x < 0:
        return ValueError('x should not be negative')

    b = a%p
    if x==0:
        return 1;
    x_list=[]
    while x!=1:
        x_list.append(x%2)
        x=x//2
    x_list.append(x)
    exp=b
    i=len(x_list)-2
    while i>=0:
        exp=(exp*exp)%p
        if x_list[i]==1:
            exp = (exp*b)%p
        i-=1

    return exp

In [25]:
def Division(a,b):
    A, B = max(a,b), min(a,b)
    q, r = divmod(A, B)
    return B, q, r

def ExtendedEuclidean(a,b):
    A,B=a,b
    if a<0:
        A=-a
    if b<0:
        B=-b
    r1, r2 = A, B
    s1, s2 = 1, 0
    t1, t2 = 0, 1
    if a==0 and b==0:
        return 0, 0, 0
    elif a==0:
        return B, 0, b/B
    elif b==0:
        return A, a/A, 0
    else:
        while r2>0:
            r1, q, r2 = Division(r1,r2)
            s=s1-q*s2
            t=t1-q*t2
            s1,s2,t1,t2 = s2,s,t2,t
        return r1, (a/A)*s1, (b/B)*t1


In [29]:
def Inv(a,p):
    a=a%p
    if p<0:
        return ValueError("p>0")
    if ExtendedEuclidean(a,p)[0] != 1:
        return ValueError("Do not exist inverse.")
    x=0
    for x in range(p):
        if (a*x)%p==1:
            break
    return x

def CRT_general(a,p):
    if type(a)==int or type(p)==int:
        return TypeError('Inputs must be lists.')
    if len(a) != len(p):
        return ValueError("The number of a must equal the number of p.")
    N=1
    for i in range(len(p)):
        N*=p[i]
    for i in range(len(p)):
       if ExtendedEuclidean(p[i],N//p[i])[0] != 1:
            return ValueError("p's should be relatively primes")
    x=0
    for i in range(len(p)):
        x+=Inv(N//p[i],p[i])*(N//p[i])*a[i]
    x=x%N

    return x

In [ ]:
from Crypto.Random import random as c_random
def MillerRabin(n, l):
    if n==1:
        return False
    if n==2:
        return True
    for ch in range(l):
        m=n-1
        k=0
        while m%2==0:
            m=m//2
            k+=1
        a=c_random.randint(2,n-1)
        b=ModExp(a,m,n)
        if b==1 or b==n-1:
            continue
        for i in range(1,k):
            b=(b*b)%n
            if b==1:
                return False
            elif b==n-1:
                break
        if b%n!=n-1:
            return False
    return True

In [ ]:
from Crypto.Random import random as c_random
def Prime_Gen(bitlength,tol):
    while True:
        prime=c_random.getrandbits(bitlength)
        if MillerRabin(prime, tol):
            return prime
def RSA_setup(bitlength,tol):
    p=Prime_Gen(bitlength,tol)
    q=Prime_Gen(bitlength,tol)
    while p==q:
        p=Prime_Gen(bitlength,tol)
        q=Prime_Gen(bitlength,tol)  
    N=p*q
    phi=(p-1)*(q-1)
    e=1
    while 1==1:
        e=c_random.randint(2,N-1)
        if ExtendedEuclidean(e,phi)[0]==1:
            break
    d=Inv(e,phi)
    pk=[N,e]
    sk=[N,d]
    return pk,sk,p,q
def RSA_Enc(pk,m):
    c=ModExp(m,pk[1],pk[0])
    return c
def RSA_Dec(sk,c):
    m=ModExp(c,sk[1],sk[0])
    return m